In [1]:
import torch

In [2]:
import pandas as pd
from sampo.scheduler.selection.neural_net import NeuralNetTrainer, NeuralNet
from sklearn.preprocessing import StandardScaler

train_dataset = pd.read_csv('dataset_mod.csv', index_col='index')
for col in train_dataset.columns[:-1]:
    train_dataset[col] = train_dataset[col].apply(lambda x: float(x))

In [3]:
input_parameters = 13
layer_size = 15
layer_count = 6
classification_size = 2
learning_rate = 0.007

model = NeuralNet(input_parameters, layer_size, layer_count, classification_size)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

trainer = NeuralNetTrainer(model, criterion, optimizer)

In [4]:
type(train_dataset)

pandas.core.frame.DataFrame

In [5]:
scaler = StandardScaler()
scaler.fit(train_dataset.drop(columns=['label']))
scaled_dataset = scaler.transform(train_dataset.drop(columns=['label']))
scaled_dataset = pd.DataFrame(scaled_dataset, columns=train_dataset.drop(columns=['label']).columns)
train_dataset = pd.concat([scaled_dataset, train_dataset['label']], axis=1)
train_dataset

,0,1,2,3,4,5,6,7,8,9,10,11,12,label
0,-0.431248,0.163774,-0.439644,0.078385,-0.100235,-0.100235,0.071216,0.184498,0.406744,-0.876387,0.045492,0.741346,-0.142857,0
1,-0.142899,0.535020,-0.768132,0.375618,-0.236551,-0.236551,-0.172504,-0.101004,-0.226645,0.575085,-0.126582,0.457406,-0.142857,0
2,-2.139163,0.668828,2.280655,0.712180,2.523848,2.523848,2.386550,2.300270,-2.081261,-2.031834,0.002473,-2.098049,-0.142857,0
3,-0.076357,-0.140131,-0.762575,0.050138,-0.264224,-0.264224,-0.221980,-0.457958,0.748588,-0.164149,-0.161514,0.399764,-0.142857,0
4,-0.142899,0.535020,-0.768132,0.375618,-0.236551,-0.236551,-0.172504,-0.101004,-0.226645,0.575085,-0.126582,0.457406,-0.142857,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.189811,-1.035411,-0.610118,-0.982248,-0.363466,-0.363466,-0.399415,-0.257116,0.815440,-0.555008,-0.286788,0.193049,-0.142857,1
1996,1.298847,-1.360059,-0.373946,-1.228777,-0.645499,-0.645499,-0.903662,-1.039082,0.111741,2.102949,-0.298655,-0.394412,-0.142857,1
1997,1.520654,-1.368336,-0.162431,-1.464193,-0.685397,-0.685397,-0.974994,-0.808293,0.190465,1.592419,-0.365806,-0.477516,-0.142857,1
1998,0.677787,-0.059682,1.108385,-0.099326,-0.508367,-0.508367,-0.658483,-0.860796,1.591849,-0.441814,-0.067850,-0.108772,-0.142857,1


In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_dataset.drop(columns=['label']), train_dataset['label'])

In [8]:
from sampo.scheduler.selection.validation import cross_val_score
from sklearn.metrics import accuracy_score

s = cross_val_score(X=x_train,
                    y=y_train,
                    scorer=accuracy_score,
                    model=trainer,
                    epochs=100,
                    folds=5,
                    shuffle=True)

Epoch [1/100], Step [100/1200], Loss: 0.6206
Epoch [1/100], Step [200/1200], Loss: 0.6206
Epoch [1/100], Step [300/1200], Loss: 0.6359
Epoch [1/100], Step [400/1200], Loss: 0.7534
Epoch [1/100], Step [500/1200], Loss: 0.6375
Epoch [1/100], Step [600/1200], Loss: 0.6340
Epoch [1/100], Step [700/1200], Loss: 0.6206
Epoch [1/100], Step [800/1200], Loss: 0.7533
Epoch [1/100], Step [900/1200], Loss: 0.7563
Epoch [1/100], Step [1000/1200], Loss: 0.6208
Epoch [1/100], Step [1100/1200], Loss: 0.6208
Epoch [1/100], Step [1200/1200], Loss: 0.7504
Epoch [2/100], Step [100/1200], Loss: 0.6206
Epoch [2/100], Step [200/1200], Loss: 0.6206
Epoch [2/100], Step [300/1200], Loss: 0.6362
Epoch [2/100], Step [400/1200], Loss: 0.7533
Epoch [2/100], Step [500/1200], Loss: 0.6377
Epoch [2/100], Step [600/1200], Loss: 0.6337
Epoch [2/100], Step [700/1200], Loss: 0.6206
Epoch [2/100], Step [800/1200], Loss: 0.7533
Epoch [2/100], Step [900/1200], Loss: 0.7554
Epoch [2/100], Step [1000/1200], Loss: 0.6206
Epoch 

2023-09-18 11:01:22,145	WARNING session.py:100 -- In neither tune session nor train session!


Epoch [100/100], Step [1100/1200], Loss: 0.3178
Epoch [100/100], Step [1200/1200], Loss: 0.7017
Epoch [1/100], Step [100/1200], Loss: 0.3150
Epoch [1/100], Step [200/1200], Loss: 0.7517
Epoch [1/100], Step [300/1200], Loss: 0.6116
Epoch [1/100], Step [400/1200], Loss: 0.6220
Epoch [1/100], Step [500/1200], Loss: 0.6238
Epoch [1/100], Step [600/1200], Loss: 0.6339
Epoch [1/100], Step [700/1200], Loss: 0.7557
Epoch [1/100], Step [800/1200], Loss: 0.6227
Epoch [1/100], Step [900/1200], Loss: 0.7185
Epoch [1/100], Step [1000/1200], Loss: 0.6383
Epoch [1/100], Step [1100/1200], Loss: 0.6337
Epoch [1/100], Step [1200/1200], Loss: 0.7184
Epoch [2/100], Step [100/1200], Loss: 0.3149
Epoch [2/100], Step [200/1200], Loss: 0.7519
Epoch [2/100], Step [300/1200], Loss: 0.6111
Epoch [2/100], Step [400/1200], Loss: 0.6220
Epoch [2/100], Step [500/1200], Loss: 0.6237
Epoch [2/100], Step [600/1200], Loss: 0.6328
Epoch [2/100], Step [700/1200], Loss: 0.7558
Epoch [2/100], Step [800/1200], Loss: 0.6227
E

2023-09-18 11:03:22,665	WARNING session.py:100 -- In neither tune session nor train session!


Epoch [100/100], Step [1100/1200], Loss: 0.5313
Epoch [100/100], Step [1200/1200], Loss: 0.6885
Epoch [1/100], Step [100/1200], Loss: 0.8500
Epoch [1/100], Step [200/1200], Loss: 0.5999
Epoch [1/100], Step [300/1200], Loss: 0.4232
Epoch [1/100], Step [400/1200], Loss: 0.5397
Epoch [1/100], Step [500/1200], Loss: 0.6208
Epoch [1/100], Step [600/1200], Loss: 0.5962
Epoch [1/100], Step [700/1200], Loss: 0.7555
Epoch [1/100], Step [800/1200], Loss: 0.7555
Epoch [1/100], Step [900/1200], Loss: 0.6210
Epoch [1/100], Step [1000/1200], Loss: 0.3135
Epoch [1/100], Step [1100/1200], Loss: 0.3136
Epoch [1/100], Step [1200/1200], Loss: 0.6533
Epoch [2/100], Step [100/1200], Loss: 0.8510
Epoch [2/100], Step [200/1200], Loss: 0.5991
Epoch [2/100], Step [300/1200], Loss: 0.4227
Epoch [2/100], Step [400/1200], Loss: 0.5388
Epoch [2/100], Step [500/1200], Loss: 0.6208
Epoch [2/100], Step [600/1200], Loss: 0.5955
Epoch [2/100], Step [700/1200], Loss: 0.7555
Epoch [2/100], Step [800/1200], Loss: 0.7555
E

...

Epoch [100/100], Step [1100/1200], Loss: 0.3133
Epoch [100/100], Step [1200/1200], Loss: 0.6101
Epoch [1/100], Step [100/1200], Loss: 0.6347
Epoch [1/100], Step [200/1200], Loss: 0.6321
Epoch [1/100], Step [300/1200], Loss: 0.6455
Epoch [1/100], Step [400/1200], Loss: 0.7494
Epoch [1/100], Step [500/1200], Loss: 0.6099
Epoch [1/100], Step [600/1200], Loss: 0.3133
Epoch [1/100], Step [700/1200], Loss: 0.3135
Epoch [1/100], Step [800/1200], Loss: 0.6091
Epoch [1/100], Step [900/1200], Loss: 0.7496
Epoch [1/100], Step [1000/1200], Loss: 0.6696
Epoch [1/100], Step [1100/1200], Loss: 0.5252
Epoch [1/100], Step [1200/1200], Loss: 0.6097
Epoch [2/100], Step [100/1200], Loss: 0.6347
Epoch [2/100], Step [200/1200], Loss: 0.6321
Epoch [2/100], Step [300/1200], Loss: 0.6451
Epoch [2/100], Step [400/1200], Loss: 0.7499
Epoch [2/100], Step [500/1200], Loss: 0.6095
Epoch [2/100], Step [600/1200], Loss: 0.3133
Epoch [2/100], Step [700/1200], Loss: 0.3134
Epoch [2/100], Step [800/1200], Loss: 0.6090
E

2023-09-18 11:07:06,912	WARNING session.py:100 -- In neither tune session nor train session!


Epoch [1/100], Step [100/1200], Loss: 0.6373
Epoch [1/100], Step [200/1200], Loss: 0.4146
Epoch [1/100], Step [300/1200], Loss: 0.4146
Epoch [1/100], Step [400/1200], Loss: 0.4554
Epoch [1/100], Step [500/1200], Loss: 0.7544
Epoch [1/100], Step [600/1200], Loss: 0.7830
Epoch [1/100], Step [700/1200], Loss: 0.4965
Epoch [1/100], Step [800/1200], Loss: 0.9982
Epoch [1/100], Step [900/1200], Loss: 0.6278
Epoch [1/100], Step [1000/1200], Loss: 0.7830
Epoch [1/100], Step [1100/1200], Loss: 0.6081
Epoch [1/100], Step [1200/1200], Loss: 0.5791
Epoch [2/100], Step [100/1200], Loss: 0.6373
Epoch [2/100], Step [200/1200], Loss: 0.4143
Epoch [2/100], Step [300/1200], Loss: 0.4143
Epoch [2/100], Step [400/1200], Loss: 0.4549
Epoch [2/100], Step [500/1200], Loss: 0.7545
Epoch [2/100], Step [600/1200], Loss: 0.7830
Epoch [2/100], Step [700/1200], Loss: 0.4962
Epoch [2/100], Step [800/1200], Loss: 0.9989
Epoch [2/100], Step [900/1200], Loss: 0.6275
Epoch [2/100], Step [1000/1200], Loss: 0.7830
Epoch 

2023-09-18 11:09:00,799	WARNING session.py:100 -- In neither tune session nor train session!


Epoch [100/100], Step [1100/1200], Loss: 0.5998
Epoch [100/100], Step [1200/1200], Loss: 0.5519


In [9]:
index = [-10000] * input_parameters
for fold in list(model.parameters())[0]:
    for i in range(len(fold)):
        index[i] = max(index[i], fold[i])
for i in range(len(index)):
    print(i, index[i])

0 tensor(1184.8458, grad_fn=<SelectBackward0>)
1 tensor(41.6979, grad_fn=<SelectBackward0>)
2 tensor(962.2628, grad_fn=<SelectBackward0>)
3 tensor(148.0795, grad_fn=<SelectBackward0>)
4 tensor(372.2423, grad_fn=<SelectBackward0>)
5 tensor(372.1004, grad_fn=<SelectBackward0>)
6 tensor(864.6303, grad_fn=<SelectBackward0>)
7 tensor(715.7028, grad_fn=<SelectBackward0>)
8 tensor(399.7072, grad_fn=<SelectBackward0>)
9 tensor(59.6007, grad_fn=<SelectBackward0>)
10 tensor(317.9749, grad_fn=<SelectBackward0>)
11 tensor(149.2861, grad_fn=<SelectBackward0>)
12 tensor(41.6908, grad_fn=<SelectBackward0>)


In [10]:
print(s)

0.8233333333333334


In [11]:
test_dataset = x_test

In [13]:
predicted = trainer.predict([torch.Tensor(v) for v in test_dataset.iloc[:, :].values])
array = []
label_test = y_test.to_numpy()
for i in range(len(predicted)):
    flag = 0 if predicted[i][0] > predicted[i][1] else 1
    array.append(int(flag == label_test[i]))
sum(array) / len(array)

0.744

In [50]:
# from sklearn.model_selection import GridSearchCV
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sampo.scheduler.selection.neural_net import NeuralNet
# from sklearn.metrics import accuracy_score
# import torch
# import numpy as np
#
# data = pd.read_csv('dataset_mod.csv', index_col='index')
# gs_params = {
#     'layer_size': [7, 15],
#     'layer_count': [3, 6],
#     'learning_rate': [0.1, 0.01]
# }

In [58]:

# from sampo.scheduler.selection.metrics import one_hot_encode
#
# x_train, x_test, y_train, y_test = train_test_split(data.drop(columns=['label']), data['label'])
# x_train = [torch.Tensor(v) for v in x_train.values]
# x_test = [torch.Tensor(v) for v in x_test.values]
# y_train = [torch.Tensor(one_hot_encode(v, 2)) for v in y_train.values]
# y_test = [torch.Tensor(one_hot_encode(v, 2)) for v in y_test.values]

In [59]:
# y_train

[tensor([1., 0.]),
 tensor([0., 1.]),
 tensor([1., 0.]),
 tensor([1., 0.]),
 tensor([1., 0.]),
 tensor([1., 0.]),
 tensor([0., 1.]),
 tensor([0., 1.]),
 tensor([1., 0.]),
 tensor([0., 1.]),
 tensor([0., 1.]),
 tensor([0., 1.]),
 tensor([1., 0.]),
 tensor([0., 1.]),
 tensor([0., 1.]),
 tensor([1., 0.]),
 tensor([0., 1.]),
 tensor([0., 1.]),
 tensor([0., 1.]),
 tensor([1., 0.]),
 tensor([0., 1.]),
 tensor([1., 0.]),
 tensor([1., 0.]),
 tensor([0., 1.]),
 tensor([0., 1.]),
 tensor([1., 0.]),
 tensor([1., 0.]),
 tensor([1., 0.]),
 tensor([0., 1.]),
 tensor([0., 1.]),
 tensor([1., 0.]),
 tensor([1., 0.]),
 tensor([0., 1.]),
 tensor([1., 0.]),
 tensor([0., 1.]),
 tensor([0., 1.]),
 tensor([0., 1.]),
 tensor([0., 1.]),
 tensor([1., 0.]),
 tensor([0., 1.]),
 tensor([1., 0.]),
 tensor([0., 1.]),
 tensor([1., 0.]),
 tensor([0., 1.]),
 tensor([1., 0.]),
 tensor([0., 1.]),
 tensor([1., 0.]),
 tensor([1., 0.]),
 tensor([0., 1.]),
 tensor([1., 0.]),
 tensor([0., 1.]),
 tensor([1., 0.]),
 tensor([0.,

In [60]:
# len(x_train), len(x_train[0])

(1500, 13)

In [61]:
# def accuracy(_estimator, _x_test, _y_test):
#     return accuracy_score(_y_test, _estimator.predict(_x_test))

In [62]:
# estimator = NeuralNet()
# gs = GridSearchCV(estimator, gs_params, cv=5, scoring=accuracy)
# gs.fit(x_train, y_train)

Epoch [1/10], Step [100/1200], Loss: 0.7384
Epoch [1/10], Step [200/1200], Loss: 0.6882
Epoch [1/10], Step [300/1200], Loss: 0.6806
Epoch [1/10], Step [400/1200], Loss: 0.7202
Epoch [1/10], Step [500/1200], Loss: 0.7130
Epoch [1/10], Step [600/1200], Loss: 0.7051
Epoch [1/10], Step [700/1200], Loss: 0.6933
Epoch [1/10], Step [800/1200], Loss: 0.7136
Epoch [1/10], Step [900/1200], Loss: 0.7167
Epoch [1/10], Step [1000/1200], Loss: 0.7140
Epoch [1/10], Step [1100/1200], Loss: 0.7230
Epoch [1/10], Step [1200/1200], Loss: 0.7087
Epoch [2/10], Step [100/1200], Loss: 0.6608
Epoch [2/10], Step [200/1200], Loss: 0.7047
Epoch [2/10], Step [300/1200], Loss: 0.6766
Epoch [2/10], Step [400/1200], Loss: 0.7130
Epoch [2/10], Step [500/1200], Loss: 0.7119
Epoch [2/10], Step [600/1200], Loss: 0.6518
Epoch [2/10], Step [700/1200], Loss: 0.6440
Epoch [2/10], Step [800/1200], Loss: 0.7132
Epoch [2/10], Step [900/1200], Loss: 0.7262
Epoch [2/10], Step [1000/1200], Loss: 0.7133
Epoch [2/10], Step [1100/120

C:\SAMPO\venv\lib\site-packages\sklearn\utils\_array_api.py:380: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  array = numpy.asarray(array, order=order, dtype=dtype)
C:\SAMPO\venv\lib\site-packages\sklearn\utils\_array_api.py:380: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the ty

Epoch [1/10], Step [100/1200], Loss: 0.6181
Epoch [1/10], Step [200/1200], Loss: 0.7478
Epoch [1/10], Step [300/1200], Loss: 0.6278
Epoch [1/10], Step [400/1200], Loss: 0.6535
Epoch [1/10], Step [500/1200], Loss: 0.6778
Epoch [1/10], Step [600/1200], Loss: 0.5879
Epoch [1/10], Step [700/1200], Loss: 0.5871
Epoch [1/10], Step [800/1200], Loss: 0.6882
Epoch [1/10], Step [900/1200], Loss: 0.7009
Epoch [1/10], Step [1000/1200], Loss: 0.6856
Epoch [1/10], Step [1100/1200], Loss: 0.6979
Epoch [1/10], Step [1200/1200], Loss: 0.6972
Epoch [2/10], Step [100/1200], Loss: 0.6733
Epoch [2/10], Step [200/1200], Loss: 0.6535
Epoch [2/10], Step [300/1200], Loss: 0.6669
Epoch [2/10], Step [400/1200], Loss: 0.6857
Epoch [2/10], Step [500/1200], Loss: 0.7294
Epoch [2/10], Step [600/1200], Loss: 0.5844
Epoch [2/10], Step [700/1200], Loss: 0.5802
Epoch [2/10], Step [800/1200], Loss: 0.7223
Epoch [2/10], Step [900/1200], Loss: 0.7407
Epoch [2/10], Step [1000/1200], Loss: 0.7154
Epoch [2/10], Step [1100/120

C:\SAMPO\venv\lib\site-packages\sklearn\utils\_array_api.py:380: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  array = numpy.asarray(array, order=order, dtype=dtype)
C:\SAMPO\venv\lib\site-packages\sklearn\utils\_array_api.py:380: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the ty

Epoch [1/10], Step [100/1200], Loss: 0.7356
Epoch [1/10], Step [200/1200], Loss: 0.6224
Epoch [1/10], Step [300/1200], Loss: 0.7404
Epoch [1/10], Step [400/1200], Loss: 0.7037
Epoch [1/10], Step [500/1200], Loss: 0.6446
Epoch [1/10], Step [600/1200], Loss: 0.6418
Epoch [1/10], Step [700/1200], Loss: 0.7034
Epoch [1/10], Step [800/1200], Loss: 0.7683
Epoch [1/10], Step [900/1200], Loss: 0.7817
Epoch [1/10], Step [1000/1200], Loss: 0.7702
Epoch [1/10], Step [1100/1200], Loss: 0.7862
Epoch [1/10], Step [1200/1200], Loss: 0.7862
Epoch [2/10], Step [100/1200], Loss: 0.7689
Epoch [2/10], Step [200/1200], Loss: 0.5793
Epoch [2/10], Step [300/1200], Loss: 0.7769
Epoch [2/10], Step [400/1200], Loss: 0.7044
Epoch [2/10], Step [500/1200], Loss: 0.6204
Epoch [2/10], Step [600/1200], Loss: 0.6186
Epoch [2/10], Step [700/1200], Loss: 0.7036
Epoch [2/10], Step [800/1200], Loss: 0.7930
Epoch [2/10], Step [900/1200], Loss: 0.8083
Epoch [2/10], Step [1000/1200], Loss: 0.7893
Epoch [2/10], Step [1100/120


KeyboardInterrupt



sklearn.model_selection._search.GridSearchCV